In [8]:
import os
import numpy as np
import pickle
from dotenv import dotenv_values
config = dotenv_values(".env")
data_path = os.path.normpath(config['DATA_PATH'])
os.environ['DATA_PATH'] = data_path
print(f"DATA_PATH is set to: {os.environ['DATA_PATH']}")

from helpers import *


DATA_PATH is set to: /mnt/big_gulp/nc4_rat_data/Maze_Rats


In [9]:
rat_number = 19
date = '250310'

msg_list = get_log(rat_number, date)
msg_list.sort(key=lambda x: x[0])
msg_list


Using path: /mnt/big_gulp/nc4_rat_data/Maze_Rats
Using path: /mnt/big_gulp/nc4_rat_data/Maze_Rats
Looking in folder: /mnt/big_gulp/nc4_rat_data/Maze_Rats/NC40019
Looking in folder: /mnt/big_gulp/nc4_rat_data/Maze_Rats/NC40019/250310/Raw/ROS


[[1741655754646575058, 'Subscribing to /Esmacat_read_sync_ease'],
 [1741655754646586498, 'Subscribing to /Esmacat_read_maze_ease'],
 [1741655754646590488, 'Subscribing to /Esmacat_read_gantry_ease'],
 [1741655754646592428, 'Subscribing to /natnet_ros/Gantry/pose'],
 [1741655754646595798, 'Subscribing to /natnet_ros/Gantry/marker0/pose'],
 [1741655754646597408, 'Subscribing to /natnet_ros/Gantry/marker1/pose'],
 [1741655754646598818, 'Subscribing to /natnet_ros/Gantry/marker2/pose'],
 [1741655754646602188, 'Subscribing to /natnet_ros/Harness/pose'],
 [1741655754646609678, 'Subscribing to /natnet_ros/Harness/marker0/pose'],
 [1741655754646612318, 'Subscribing to /natnet_ros/Harness/marker1/pose'],
 [1741655754646618568, 'Subscribing to /natnet_ros/Harness/marker2/pose'],
 [1741655754646620348, 'Subscribing to /natnet_ros/MazeBoundary/pose'],
 [1741655754646631808, 'Subscribing to /natnet_ros/MazeBoundary/marker0/pose'],
 [1741655754646635248, 'Subscribing to /natnet_ros/MazeBoundary/mark

In [10]:
# Filter messages
trial_start_msg_list = [msg for msg in msg_list if msg[1].startswith('Current trial number:')]
trial_start_msg_list

trial_end_msg_list = [msg for msg in msg_list if msg[1].startswith('INTER_TRIAL_INTERVAL')]
trial_end_msg_list



[[1741655944806720345, 'INTER_TRIAL_INTERVAL'],
 [1741656027126281166, 'INTER_TRIAL_INTERVAL'],
 [1741656095566304801, 'INTER_TRIAL_INTERVAL'],
 [1741656139955961842, 'INTER_TRIAL_INTERVAL'],
 [1741656200385988140, 'INTER_TRIAL_INTERVAL'],
 [1741656236955894697, 'INTER_TRIAL_INTERVAL'],
 [1741656269026574683, 'INTER_TRIAL_INTERVAL'],
 [1741656305956162137, 'INTER_TRIAL_INTERVAL'],
 [1741656354066268524, 'INTER_TRIAL_INTERVAL'],
 [1741656391866220345, 'INTER_TRIAL_INTERVAL'],
 [1741656430426535506, 'INTER_TRIAL_INTERVAL'],
 [1741656466416393934, 'INTER_TRIAL_INTERVAL'],
 [1741656523336471547, 'INTER_TRIAL_INTERVAL'],
 [1741656556456829387, 'INTER_TRIAL_INTERVAL'],
 [1741656615466425468, 'INTER_TRIAL_INTERVAL'],
 [1741656657116292945, 'INTER_TRIAL_INTERVAL'],
 [1741656709686194659, 'INTER_TRIAL_INTERVAL'],
 [1741656740756568388, 'INTER_TRIAL_INTERVAL'],
 [1741656768066076947, 'INTER_TRIAL_INTERVAL'],
 [1741656799066648056, 'INTER_TRIAL_INTERVAL'],
 [1741656836156557546, 'INTER_TRIAL_INTE

In [24]:
from collections import deque
import pprint

# Define exclusion conditions
exclusion_conditions = [
    {
        "choice_correctness": "Choice is incorrect: 0",
        "correct_choice": "Correct stimulus-response choice is: 1",
        "gantry_sequence": ["4", "3", "0", "3"]

    },
    {
        "choice_correctness": "Choice is incorrect: 0",
        "correct_choice": "Correct stimulus-response choice is: 2",
        "gantry_sequence": ["4", "3", "6", "3"]

    },
    {
        "choice_correctness": "Choice is correct: 1",
        "correct_choice": "Correct stimulus-response choice is: 2",
        "gantry_sequence": ["4", "3", "0", "3"]

    },
    {
        "choice_correctness": "Choice is correct: 1",
        "correct_choice": "Correct stimulus-response choice is: 1",
        "gantry_sequence": ["4", "3", "6", "3"]

    }
]

used_trial_numbers = set() # Track used trial numbers to prevent duplicates
used_trial_start_times = set() # Track used trial start times to avoid reusing trial starts

trials = []
for msg in trial_end_msg_list:
    # Find the closest previous trial start message
    trial_start_msg_candidates = [msg_start for msg_start in trial_start_msg_list if msg_start[0] < msg[0] and msg_start[0] not in used_trial_start_times]
    if not trial_start_msg_candidates:
        continue # Skip if no valid start message found

    trial_start_msg = trial_start_msg_candidates[-1]
    trial_start_time = trial_start_msg[0]
    trial_end_time = msg[0]

    # Mark this start time to prevent duplicates
    used_trial_start_times.add(trial_start_time)

    # Extract trial number & timestamps
    trial_number_msg = next((m for m in msg_list if m[1].startswith("Current trial number:") and trial_start_time <= m[0] < trial_end_time), None)

    if trial_number_msg:
        trial_number = int(trial_number_msg[1].split(": ")[1]) + 1
        if trial_number in used_trial_numbers:
            continue # Skip duplicates
        used_trial_numbers.add(trial_number)
    else:
        trial_number = None

    #trial_number = int(trial_number_msg[1].split(": ")[1]) + 1 if trial_number_msg else None

    # Dynamically filter messages within each trial window
    trial_msgs = [m for m in msg_list if trial_start_time <= m[0] < trial_end_time]


    ## Extract key message within the trial
    # Extract choice correctnmess message
    choice_correctness_msg = next((m for m in trial_msgs if "Choice is incorrect: 1" in m[1] or "Choice is incorrect: 0" in m[1]), None)
    
    #Extract correct stimulus-response choice
    correct_choice_msg = next((m for m in trial_msgs if ("Correct stimulus-response choice is:") in m[1]), None)

    # Extract actual choice (y)
    choice_y_msg = next((m for m in trial_msgs if m[1].startswith("Choice (y) is")), None)

    # Extract stimulus value
    stimulus_msg = next((m for m in trial_msgs if "Selected stimulus is:" in m[1]), None)

    # Extract the "CHOICE" message
    choice_msg = next ((m for m in trial_msgs if m[1] == "CHOICE"), None)

    if choice_msg is None:
        continue # Skip trial is no CHOICE message found

    # Extract gantry movement AFTER "CHOICE"
    gantry_messages_after_choice = []
    choice_index = msg_list.index(choice_msg)

    for m in trial_msgs[choice_index:]:
        if "Move to chamber command received: chamber[" in m[1]:
            chamber = m[1].split("chamber[")[1].split("]")[0]
            gantry_messages_after_choice.append(chamber)
        if len(gantry_messages_after_choice) == 4: # Stop after 4 messages
            break
    
    # Extract actual values
    choice_correctness = 1 if choice_correctness_msg and "Choice is incorrect: 1" in choice_correctness_msg[1] else 0
    correct_choice = int(correct_choice_msg[1].split(": ")[1]) if correct_choice_msg else None
    choice_y = int(choice_y_msg[1].split()[-1]) if choice_y_msg else None
    stimulus = int(stimulus_msg[1].split(": ")[1]) if stimulus_msg else None

    for condition in exclusion_conditions:
        if (
            choice_correctness == condition["choice_correctness"]
            and correct_choice == condition["correct_choice"]
            and gantry_messages_after_choice == condition["gantry_sequence"]
        ):
            print(f"Skipping trial (trial_number) due to missed trial")
            break # Skip this trial

    else:
        trials.append( {
            "trial_number": trial_number,
            "trial_start": trial_start_time,
            "trial_end": trial_end_time,
            "y": choice_y,
            "answer": correct_choice,
            "correct": choice_correctness,
            "stimulus": stimulus,
            "gantry_sequence": gantry_messages_after_choice
        })

    print(f"Processed {len(trials)} valid trials after filtering.")
    pprint.pprint(trials[:4])

Processed 1 valid trials after filtering.
[{'answer': 1,
  'correct': 0,
  'gantry_sequence': [],
  'stimulus': 1,
  'trial_end': 1741655944806720345,
  'trial_number': 1,
  'trial_start': 1741655773066424502,
  'y': None}]
Processed 2 valid trials after filtering.
[{'answer': 1,
  'correct': 0,
  'gantry_sequence': [],
  'stimulus': 1,
  'trial_end': 1741655944806720345,
  'trial_number': 1,
  'trial_start': 1741655773066424502,
  'y': None},
 {'answer': 2,
  'correct': 0,
  'gantry_sequence': [],
  'stimulus': -1,
  'trial_end': 1741656027126281166,
  'trial_number': 2,
  'trial_start': 1741655945826962126,
  'y': None}]
Processed 3 valid trials after filtering.
[{'answer': 1,
  'correct': 0,
  'gantry_sequence': [],
  'stimulus': 1,
  'trial_end': 1741655944806720345,
  'trial_number': 1,
  'trial_start': 1741655773066424502,
  'y': None},
 {'answer': 2,
  'correct': 0,
  'gantry_sequence': [],
  'stimulus': -1,
  'trial_end': 1741656027126281166,
  'trial_number': 2,
  'trial_start

In [ ]:
# Covert extracted data into NumPy arrays
y = np.array([int(m[1].split("Choice (y) is ")[-1]) for m in msg_list if "Choice (y) is" in m[1]])
answer = np.array([int(m[1].split("Correct stimulus-response choice is: ")[-1]) for m in msg_list if "Correct stimulus-response choice is:" in m[1]])
correct = np.array([1 if "Choice is correct: 1" in m[1] else 0 for m in msg_list if "Choice is" in m[1]])

# Define history length for inputs
history_length = 3

# Construct 'inputs' dictionary 
stimulus_values = np.array([t["gantry_sequence"] for t in trials])
stimulus_history = np.zeros((len(stimulus_values), history_length))
for i in range(len(stimulus_values)):
    for j in range(history_length):
        if i - j >= 0:
            stimulus_history[i, j] = stimulus_values[i - j]

inputs = {"s1": stimulus_history}

print(f"Constructed input matrix with shape {stimulus_history.shape}.")

ValueError: setting an array element with a sequence.

In [ ]:
## Construct trial list
#trial_list = []
#k=1
#for msg in trial_end_msg_list:
#    # Find the corresponding trial start message whose timestamp is smaller than the trial end message
#    trial_start_msg = [msg_start for msg_start in trial_start_msg_list if msg_start[0] < msg[0]][-1]
#    trial_list.append([k, trial_start_msg[0], msg[0]])
#    k += 1

#trial_list

# Get duration of each trial
#for trial in trial_list:
#    trial.append((trial[2] - trial[1])*1.0/1e9)

#trial_list

[[1, 1741658921977109052, 1741659010747037609, 88.769928557],
 [2, 1741659011767274436, 1741659069036904557, 57.269630121],
 [3, 1741659070057380277, 1741659099717657155, 29.660276878],
 [4, 1741659100737323817, 1741659297291432715, 196.554108898],
 [5, 1741659298301973246, 1741659373703859042, 75.401885796],
 [6, 1741659374714045791, 1741659432073978910, 57.359933119],
 [7, 1741659433084077337, 1741659513784160187, 80.70008285],
 [8, 1741659514794734644, 1741659547224713164, 32.42997852],
 [9, 1741659548244279456, 1741659624674059358, 76.429779902],
 [10, 1741659625684626321, 1741659670564337701, 44.87971138],
 [11, 1741659671575174981, 1741659717273904850, 45.698729869],
 [12, 1741659718285033096, 1741659781944051968, 63.659018872],
 [13, 1741659782964346810, 1741659821254493702, 38.290146892],
 [14, 1741659822264860810, 1741659873114470059, 50.849609249],
 [15, 1741659874124979901, 1741659927994375861, 53.86939596],
 [16, 1741659929004112113, 1741659964874826663, 35.87071455],
 [17,

In [ ]:
# Find messages within a trial that contains 'Selected stimulus is:'

#selected_stimulus_msg_list = [msg for msg in msg_list if msg[1].startswith('Selected stimulus is:')]

#for msg in selected_stimulus_msg_list:
    # Find the corresponding trial
    #trial = [trial for trial in trial_list if trial[1] < msg[0] and trial[2] > msg[0]][0]
    #print(f"Trial {trial[0]}: {msg[1]}")